# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Sitka,57.0531,-135.3300,37.29,87,100,4.61,1018,US,1732703875
1,1,Bethel,41.3712,-73.4140,38.66,78,0,4.61,1016,US,1732703877
2,2,Carutapera,-1.1950,-46.0200,83.05,67,64,10.45,1012,BR,1732703878
3,3,Tongliao,43.6125,122.2653,17.37,72,98,10.02,1004,CN,1732703879
4,4,São Raimundo Nonato,-9.0153,-42.6994,79.68,70,51,9.04,1014,BR,1732703880


In [4]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     590 non-null    int64  
 1   City        590 non-null    object 
 2   Lat         590 non-null    float64
 3   Lng         590 non-null    float64
 4   Max Temp    590 non-null    float64
 5   Humidity    590 non-null    int64  
 6   Cloudiness  590 non-null    int64  
 7   Wind Speed  590 non-null    float64
 8   Pressure    590 non-null    int64  
 9   Country     589 non-null    object 
 10  Date        590 non-null    int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 50.8+ KB


In [5]:
# Date time cleaning - Option1: city_data_df["Date"] = pd.to_datetime(city_data_df.Date * 1e9)
city_data_df["Date"] = pd.to_datetime(city_data_df["Date"])
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Sitka,57.0531,-135.3300,37.29,87,100,4.61,1018,US,1970-01-01 00:00:01.732703875
1,1,Bethel,41.3712,-73.4140,38.66,78,0,4.61,1016,US,1970-01-01 00:00:01.732703877
2,2,Carutapera,-1.1950,-46.0200,83.05,67,64,10.45,1012,BR,1970-01-01 00:00:01.732703878
3,3,Tongliao,43.6125,122.2653,17.37,72,98,10.02,1004,CN,1970-01-01 00:00:01.732703879
4,4,São Raimundo Nonato,-9.0153,-42.6994,79.68,70,51,9.04,1014,BR,1970-01-01 00:00:01.732703880


In [6]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     590 non-null    int64         
 1   City        590 non-null    object        
 2   Lat         590 non-null    float64       
 3   Lng         590 non-null    float64       
 4   Max Temp    590 non-null    float64       
 5   Humidity    590 non-null    int64         
 6   Cloudiness  590 non-null    int64         
 7   Wind Speed  590 non-null    float64       
 8   Pressure    590 non-null    int64         
 9   Country     589 non-null    object        
 10  Date        590 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(2)
memory usage: 50.8+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip. No need size or scale
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 69.8
max_temp = 80.6
max_wind = 14

# Drop any rows with null values
df2=city_data_df.dropna()
# Optional: df2=df2.loc[(df2["Max Temp"]>=min_temp) & (df2["Max Temp"]<=max_temp) & (df2["Wind Speed"]<max_wind)]
mask=(df2["Max Temp"]>=min_temp) & (df2["Max Temp"]<=max_temp) & (df2["Wind Speed"]<max_wind)
df2=df2.loc[mask].reset_index(drop=True)

df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,4,São Raimundo Nonato,-9.0153,-42.6994,79.68,70,51,9.04,1014,BR,1970-01-01 00:00:01.732703880
1,13,Correntina,-13.3433,-44.6367,70.29,94,100,4.41,1015,BR,1970-01-01 00:00:01.732703890
2,15,Puerto Ayora,-0.7393,-90.3518,70.30,81,98,10.40,1013,EC,1970-01-01 00:00:01.732703892
3,20,Bhamo,24.2667,97.2333,70.05,71,97,2.57,1012,MM,1970-01-01 00:00:01.732703898
4,30,George Town,5.4112,100.3354,78.73,82,40,10.36,1008,MY,1970-01-01 00:00:01.732703910
...,...,...,...,...,...,...,...,...,...,...,...
131,569,Katabu,-4.9333,122.5167,77.86,91,90,2.33,1007,ID,1970-01-01 00:00:01.732704551
132,577,Crucecita,-28.9500,-58.9833,71.15,91,100,6.13,1007,AR,1970-01-01 00:00:01.732704560
133,578,Palāsa,18.7667,84.4167,74.35,55,92,11.21,1011,IN,1970-01-01 00:00:01.732704561
134,579,Mhamid,29.8200,-5.7200,71.10,28,34,2.86,1025,MA,1970-01-01 00:00:01.732704562


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
hotel_rows = []

for index, row in df2.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [13]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,São Raimundo Nonato,BR,-9.0153,-42.6994,"Pousada Lelinha, Rua Doutor Barroso 249, São R...",Pousada Lelinha,274.0,None,None
1,Correntina,BR,-13.3433,-44.6367,"Pousada Silva, Rua João de Castro Rego, Corren...",Pousada Silva,339.0,None,None
2,Puerto Ayora,EC,-0.7393,-90.3518,"Hostal Mirada Del Solitario George Hostel, Pet...",Hostal La Mirada De Solitario George,3776.0,None,None
3,Bhamo,MM,24.2667,97.2333,"Winlight Hotel (foreigners allowed), ဗန်းမော် ...",Winlight Hotel,880.0,None,None
4,George Town,MY,5.4112,100.3354,"Page 63 hostel, Beach Street, 10300 George Tow...",Page 63 hostel,139.0,None,None
...,...,...,...,...,...,...,...,...,...
131,Katabu,ID,-4.9333,122.5167,None,None,NaN,None,None
132,Crucecita,AR,-28.9500,-58.9833,None,None,NaN,None,None
133,Palāsa,IN,18.7667,84.4167,None,None,NaN,None,None
134,Mhamid,MA,29.8200,-5.7200,"Hotel Kasbah Azalay, RN9, 47404 M'Hamid El Ghi...",Hotel Kasbah Azalay,213.0,None,None


In [14]:
data


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'country': 'Venezuela',
    'country_code': 've',
    'state': 'Yaracuy State',
    'county': 'Municipio San Felipe',
    'city': 'San Felipe',
    'municipality': 'Parroquia San Felipe',
    'postcode': '3210',
    'street': 'Avenida La Patria',
    'lon': -68.743526,
    'lat': 10.340573,
    'formatted': 'Avenida La Patria, San Felipe 3210, Yaracuy State, Venezuela',
    'address_line1': 'Avenida La Patria',
    'address_line2': 'San Felipe 3210, Yaracuy State, Venezuela',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database License',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'osm_id': 1196828700, 'tourism': 'hotel', 'osm_type': 'n'}},
    'distance': 135,
    'place_id': '51726e13ee952f51c059fe208d955fae2440f00103f9011c

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name", "address"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name,address)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
#### Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (2718306526.py, line 4)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name", "address"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot